# Enunciado

Genere un modelo de líquido compuesto por agua y etanol en una relación de volumen 2:1 dentro de una celda cúbica de 20 Å de lado. Busque en la web los archivos XYZ para las moléculas de agua y etanol, calcule el número necesario de moléculas de cada tipo y utilice PACKMOL para construir el líquido. Configure las restricciones para evitar solapamiento y verifique el archivo resultante.

Cálculo del número de moléculas
Para determinar cuántas moléculas caben en la celda:

Volumen de la celda: $20^3 Å^3 = 8000 Å^3 $

Densidades y masas molares:

* Agua: 1.0 g/cm³, 18.015 g/mol

* Etanol: 0.789 g/cm³, 46.07 g/mol

Dado que la fracción de volumen es $\frac{2}{3}$ para agua y $\frac{1}{3}$ para etanol:

* Masa total de agua:

$$m_{agua} = (\frac{2}{3} \times 8000 Å^3) \times \frac{1 g/cm^{3}}{10^{24} Å^{3}/cm^{3}} = 5.33 \times 10 ^{-21} g$$

* Número de moléculas de agua:

$$\frac{5.33 \times 10 ^{-21} g}{18.015  \times 10 ^{-24} g/mol} \approx 296$$

* Masa total de etanol:

$$m_{etanol} = (\frac{1}{3} \times 8000 Å^3) \times \frac{0.789 g/cm^{3}}{10^{24} Å^{3}/cm^{3}} = 2.10 \times 10 ^{-21} g$$

* Número de moléculas de etanol:

$$\frac{2.10 \times 10 ^{-21} g}{46.07 \times 10^{-24} g/mol} \approx 46$$

Por lo tanto, necesitas aproximadamente 296 moléculas de agua y 46 de etanol en la celda de 20 Å³.

Se usó el archivo `packmol.inp` como input para Packmol. Para correrlo se utilizó Julia y desde el interpretador de Julia, una vez aquí se utilizó lo siguente:

```julia
using Packmol;
run_packmon()
```

De modo que se abríera la interfaz de Packmol para escoger el archivo que tenía el input correspondiente

In [1]:
from ase import Atoms
from ase.io import read, write
from ase.visualize import view # Importa la herramienta para visualizar la estructura
mol = read("mezcla.xyz")
view(mol)

<Popen: returncode: None args: ['/home/hernan-barquero/anaconda3/bin/python'...>

In [2]:
mol.cell = [[25.0,0.0,0.0],[0.0,25.0,0.0],[0.0,0.0,25.0]]
mol.positions[:,0] += 2.5
mol.positions[:,1] += 2.5
mol.positions[:,2] += 2.5
view(mol)

<Popen: returncode: None args: ['/home/hernan-barquero/anaconda3/bin/python'...>

In [3]:
# Genera un archivo de entrada para Quantum ESPRESSO con la configuración donde el hidrógeno está sobre el átomo de carbono.
write("mezcla_2agua_1etano.in", mol, format="espresso-in", 
	pseudopotentials={
    "C": "C.pbe-n-kjpaw_psl.1.0.0.UPF",  # Especifica el pseudopotencial para carbono.
    "H": "H.pbe-kjpaw_psl.1.0.0.UPF",     # Especifica el pseudopotencial para hidrógeno.
	"O": "O.pbe-n-kjpaw_psl.1.0.0.UPF"
}, input_data={
    "control": {
        "calculation": "relax",           # Indica que se hará una relajación de la estructura.
        "prefix": "2A_1E",    # Prefijo para los archivos de salida generados por Quantum ESPRESSO.
        "outdir": ".",             # Carpeta donde se guardarán los archivos temporales de Quantum ESPRESSO.
        "pseudo_dir": "/home/ebarquero/practicas/p3/pseudo/",          # Carpeta donde se encuentran los pseudopotenciales.
		"tstress": True,
		"tprnfor": True
    },
    "system": {
        "ecutwfc": 50,                    # Corte de energía para las funciones de onda (en Ry).
        "ecutrho": 330,                   # Corte de energía para la densidad de carga (en Ry).
        "occupations": "smearing",        # Usa un método de smearing para las ocupaciones electrónicas.
        "smearing": "mp",                 # Método de smearing: Methfessel-Paxton.
        "degauss": 0.02 ,                  # Ancho del smearing en Ry.
		"ibrav":0, 							# Define una celda generica
    },
    "electrons": {
        "conv_thr": 1e-6                  # Criterio de convergencia para las iteraciones electrónicas.
    }
}, kpts = (1,1,1),koffset = (0,0,0))